In [1]:
import pandas as pd
from pathlib import Path


In [2]:
[i for i in Path('Case 2 - PA').iterdir()]

[WindowsPath('Case 2 - PA/00. Case PA - Arquitetura Dados Recebidos.png'),
 WindowsPath('Case 2 - PA/ATENDIMENTO.csv'),
 WindowsPath('Case 2 - PA/FILA_SENHA.csv'),
 WindowsPath('Case 2 - PA/FILA_SENHA.xlsx'),
 WindowsPath('Case 2 - PA/MOTIVO_ALTA.csv'),
 WindowsPath('Case 2 - PA/MOTIVO_ALTA.xlsx'),
 WindowsPath('Case 2 - PA/PESSOA_FISICA_MEDICO.csv'),
 WindowsPath('Case 2 - PA/PESSOA_FISICA_MEDICO.xlsx'),
 WindowsPath('Case 2 - PA/PESSOA_FISICA_PAC.csv'),
 WindowsPath('Case 2 - PA/PESSOA_FISICA_PAC.xlsx')]

## Carregandos as bases.

In [3]:
service = pd.read_csv('Case 2 - PA/ATENDIMENTO.csv',sep=';')
queue_passw = pd.read_excel('Case 2 - PA/FILA_SENHA.xlsx')
discharge = pd.read_excel('Case 2 - PA/MOTIVO_ALTA.xlsx')
person_medic = pd.read_excel('Case 2 - PA/PESSOA_FISICA_MEDICO.xlsx')
person = pd.read_excel('Case 2 - PA/PESSOA_FISICA_PAC.xlsx')


In [4]:
# muitos dados como campo de data e outros identificadores primarios com valores nulos.

In [5]:
service.isna().mean()

NR_ATENDIMENTO           0.000000
DT_ENTRADA               0.000000
DT_INICIO_ATENDIMENTO    0.006482
DT_ATEND_MEDICO          0.015672
DT_ALTA                  0.000620
DT_FIM_TRIAGEM           0.007433
NR_SEQ_TRIAGEM           0.007588
DS_SENHA_QMATIC          0.316928
DT_MEDICACAO             0.538761
NR_SEQ_CLASSIFICACAO     0.000093
CD_MOTIVO_ALTA           0.000620
IE_TIPO_ATENDIMENTO      0.000000
CD_PESSOA_FISICA         0.000000
CD_MEDICO_RESP           0.000000
NR_SEQ_PAC_SENHA_FILA    0.030485
IE_CLINICA               0.000000
DT_RECEBIMENTO_SENHA     0.322407
dtype: float64

In [6]:
service.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96735 entries, 0 to 96734
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   NR_ATENDIMENTO         96735 non-null  int64  
 1   DT_ENTRADA             96735 non-null  object 
 2   DT_INICIO_ATENDIMENTO  96108 non-null  object 
 3   DT_ATEND_MEDICO        95219 non-null  object 
 4   DT_ALTA                96675 non-null  object 
 5   DT_FIM_TRIAGEM         96016 non-null  object 
 6   NR_SEQ_TRIAGEM         96001 non-null  float64
 7   DS_SENHA_QMATIC        66077 non-null  object 
 8   DT_MEDICACAO           44618 non-null  object 
 9   NR_SEQ_CLASSIFICACAO   96726 non-null  float64
 10  CD_MOTIVO_ALTA         96675 non-null  float64
 11  IE_TIPO_ATENDIMENTO    96735 non-null  int64  
 12  CD_PESSOA_FISICA       96735 non-null  int64  
 13  CD_MEDICO_RESP         96735 non-null  int64  
 14  NR_SEQ_PAC_SENHA_FILA  93786 non-null  float64
 15  IE

In [7]:
queue_passw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   NR_SEQUENCIA          14 non-null     int64         
 1   DT_ATUALIZACAO        14 non-null     datetime64[ns]
 2   DS_FILA               14 non-null     object        
 3   DS_LETRA_VERIFICACAO  14 non-null     object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 580.0+ bytes


In [8]:
queue_passw = queue_passw.rename(columns={"NR_SEQUENCIA":"NR_SEQ_TRIAGEM"})

In [9]:
merge1 = service.merge(queue_passw,on='NR_SEQ_TRIAGEM')

In [10]:
discharge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   CD_MOTIVO_ALTA  34 non-null     int64 
 1   DS_MOTIVO_ALTA  34 non-null     object
dtypes: int64(1), object(1)
memory usage: 676.0+ bytes


In [11]:
merge2 = merge1.merge(discharge,on='CD_MOTIVO_ALTA')

In [12]:
person_medic = person_medic.rename(columns={'CD_PESSOA_FISICA':'CD_MEDICO_RESP'})

In [13]:
person_medic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   CD_MEDICO_RESP      146 non-null    int64 
 1   NM_PESSOA_PESQUISA  146 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.4+ KB


In [14]:
merge3 = merge2.merge(person_medic, on ='CD_MEDICO_RESP')

In [15]:
person.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37077 entries, 0 to 37076
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   CD_PESSOA_FISICA  37077 non-null  int64         
 1   DT_NASCIMENTO     37077 non-null  datetime64[ns]
 2   IE_SEXO           37077 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 869.1+ KB


In [16]:
merge4 = merge3.merge(person,on='CD_PESSOA_FISICA')

In [17]:
merge4=merge4[['NR_ATENDIMENTO', 'DT_ENTRADA', 'DT_INICIO_ATENDIMENTO',
       'DT_ATEND_MEDICO', 'DT_FIM_TRIAGEM', 'NR_SEQ_TRIAGEM',
       'DS_SENHA_QMATIC', 'DT_MEDICACAO', 'NR_SEQ_CLASSIFICACAO',
       'CD_MOTIVO_ALTA','DS_MOTIVO_ALTA', 'DT_ALTA', 'IE_TIPO_ATENDIMENTO', 'CD_PESSOA_FISICA','DT_NASCIMENTO', 'IE_SEXO',
       'CD_MEDICO_RESP','NM_PESSOA_PESQUISA', 'NR_SEQ_PAC_SENHA_FILA', 'IE_CLINICA',
       'DT_RECEBIMENTO_SENHA', 'DT_ATUALIZACAO', 'DS_FILA',
       'DS_LETRA_VERIFICACAO']]
       

In [18]:
(1-(merge4.shape[0])/service.shape[0]) * 100

0.7897865302114004

## Ponto de Atenção!
Atraves dos merges 0.78% dos dados foram peridos o que indica falta de dados das chaves primarias para os joins e isso e muito
delicado. 

Por que esses dados não foram atendidos? exsite a data de entrada porem sem data de inicio

In [19]:
merge4[merge4['DT_INICIO_ATENDIMENTO'].isna()]

,NR_ATENDIMENTO,DT_ENTRADA,DT_INICIO_ATENDIMENTO,DT_ATEND_MEDICO,DT_FIM_TRIAGEM,NR_SEQ_TRIAGEM,DS_SENHA_QMATIC,DT_MEDICACAO,NR_SEQ_CLASSIFICACAO,CD_MOTIVO_ALTA,...,DT_NASCIMENTO,IE_SEXO,CD_MEDICO_RESP,NM_PESSOA_PESQUISA,NR_SEQ_PAC_SENHA_FILA,IE_CLINICA,DT_RECEBIMENTO_SENHA,DT_ATUALIZACAO,DS_FILA,DS_LETRA_VERIFICACAO
14465,108073,2022/02/05 09:31:09,NaN,2022/02/05 10:03:57,2022/02/05 09:44:32,4.0,NaN,NaN,17.0,12.0,...,1955-02-08,F,159214,HUGO EDINALDO AGUIAR,103953.0,1,NaN,2020-03-18 14:24:15,PS - Medicação/Procedimentos,CP
44294,107435,2022/01/31 15:31:00,NaN,NaN,NaN,9.0,NaN,NaN,17.0,18.0,...,1979-03-02,M,107833,ADRIANA FLÁVIA DE CALDEIRA,103015.0,1,NaN,2023-08-11 09:22:59,RI - Oncologia,O
46635,102312,2022/01/06 13:45:34,NaN,NaN,NaN,9.0,NaN,NaN,17.0,18.0,...,1972-02-21,F,107833,ADRIANA FLÁVIA DE CALDEIRA,96124.0,1,NaN,2023-08-11 09:22:59,RI - Oncologia,O
67974,107451,2022/01/31 16:43:10,NaN,NaN,NaN,9.0,NaN,NaN,17.0,18.0,...,1976-04-24,M,107833,ADRIANA FLÁVIA DE CALDEIRA,103034.0,1,NaN,2023-08-11 09:22:59,RI - Oncologia,O
83803,123866,2022/05/25 15:08:57,NaN,2022/05/25 16:34:54,2022/05/25 15:37:52,4.0,NaN,NaN,8.0,12.0,...,1983-08-29,M,205397,ALTAIR LUCAS CHAVES SANTIAGO DE OLIVEIRA,126417.0,1,NaN,2020-03-18 14:24:15,PS - Medicação/Procedimentos,CP
93071,103181,2022/01/10 15:28:17,NaN,2022/01/10 15:52:58,NaN,9.0,NaN,2022/01/10 15:46:07,8.0,12.0,...,1998-02-17,F,14189,MAGALI TÁBATA FRANCO LOVATO DE MACHADO,97202.0,10,NaN,2023-08-11 09:22:59,RI - Oncologia,O


In [20]:
df_analysis = merge4.dropna(subset=['DT_ENTRADA','DT_INICIO_ATENDIMENTO','DT_ATEND_MEDICO','DT_ALTA','DT_FIM_TRIAGEM']).copy()

In [21]:
df_analysis['DT_INICIO_ATENDIMENTO']=df_analysis['DT_INICIO_ATENDIMENTO'].astype('datetime64[ns]')
df_analysis['DT_ENTRADA']=df_analysis['DT_ENTRADA'].astype('datetime64[ns]')
df_analysis['DT_ATEND_MEDICO']=df_analysis['DT_ATEND_MEDICO'].astype('datetime64[ns]')
df_analysis['DT_ALTA']=df_analysis['DT_ALTA'].astype('datetime64[ns]')
df_analysis['DT_FIM_TRIAGEM']=df_analysis['DT_FIM_TRIAGEM'].astype('datetime64[ns]')


In [22]:
df_analysis['TEMPO ESPERA'] = (df_analysis['DT_INICIO_ATENDIMENTO'] - df_analysis['DT_ENTRADA'])/pd.Timedelta(1,'m')

In [23]:
df_analysis1 = df_analysis.sort_values('TEMPO ESPERA')[['TEMPO ESPERA','NR_ATENDIMENTO', 'DT_ENTRADA', 'DT_INICIO_ATENDIMENTO',
  'NR_SEQ_TRIAGEM',
    'DS_FILA',
       'DS_MOTIVO_ALTA', 'DT_ALTA', 'IE_TIPO_ATENDIMENTO', 'CD_PESSOA_FISICA','DT_NASCIMENTO', 'IE_SEXO',
       'CD_MEDICO_RESP','NM_PESSOA_PESQUISA', 'IE_CLINICA',
       'DS_LETRA_VERIFICACAO']]

In [24]:
df_analysis2 = df_analysis1[df_analysis1['TEMPO ESPERA'] <= 30]

In [25]:
hist2 = df_analysis2['DS_FILA'].value_counts()/df_analysis.shape[0]

In [26]:
hist2.reset_index()

,DS_FILA,count
0,PS - Medicação/Procedimentos,0.797130
1,PS - Ambulância,0.154741
2,RI - Proced. Ambulatorial,0.007839
3,RI - Oncologia,0.002904
4,PS - Médico Externo,0.001631
5,PS - Normal,0.000526


95% dos atendimentos ate meia hora sao de PS - Medicação/Procedimentos e de PS - Ambulância

In [27]:
df_analysis['TEMPO P/ ATENDIMENTO'] = (df_analysis['DT_ATEND_MEDICO'] - df_analysis['DT_ENTRADA'])/pd.Timedelta(1,'m')
df_analysis['TEMPO P/ ALTA'] = (df_analysis['DT_ALTA'] - df_analysis['DT_ATEND_MEDICO'])/pd.Timedelta(1,'m')

In [28]:
df_analysis.columns

Index(['NR_ATENDIMENTO', 'DT_ENTRADA', 'DT_INICIO_ATENDIMENTO',
       'DT_ATEND_MEDICO', 'DT_FIM_TRIAGEM', 'NR_SEQ_TRIAGEM',
       'DS_SENHA_QMATIC', 'DT_MEDICACAO', 'NR_SEQ_CLASSIFICACAO',
       'CD_MOTIVO_ALTA', 'DS_MOTIVO_ALTA', 'DT_ALTA', 'IE_TIPO_ATENDIMENTO',
       'CD_PESSOA_FISICA', 'DT_NASCIMENTO', 'IE_SEXO', 'CD_MEDICO_RESP',
       'NM_PESSOA_PESQUISA', 'NR_SEQ_PAC_SENHA_FILA', 'IE_CLINICA',
       'DT_RECEBIMENTO_SENHA', 'DT_ATUALIZACAO', 'DS_FILA',
       'DS_LETRA_VERIFICACAO', 'TEMPO ESPERA', 'TEMPO P/ ATENDIMENTO',
       'TEMPO P/ ALTA'],
      dtype='object')

In [29]:
df_tempos = df_analysis.groupby(by=['DS_MOTIVO_ALTA','CD_PESSOA_FISICA'])[['TEMPO ESPERA', 'TEMPO P/ ATENDIMENTO',
       'TEMPO P/ ALTA']].mean()

###  tempo por cadastro de pessoa e motivo alta

In [30]:
df_tempos

TEMPO ESPERA  \
DS_MOTIVO_ALTA                         CD_PESSOA_FISICA                 
Alta Curado                            21726                 2.783333   
                                       32416                11.666667   
                                       46850                 5.300000   
                                       83799                 5.016667   
                                       93614                13.233333   
...                                                               ...   
Transferido para outro Estabelecimento 216866                9.066667   
                                       217950                8.033333   
                                       218728                5.250000   
                                       218800               23.833333   
                                       219513                4.533333   

                                                         TEMPO P/ ATENDIMENTO  \
DS_MOTIVO_ALTA                         CD_PESSOA_FISICA                         
Alta Curado                            21726                        35.583333   
                                       32416                        23.116667   
                                       46850                        48.066667   
                                       83799                        10.250000   
                                       93614                        56.283333   
...                                                                       ...   
Transferido para outro Estabelecimento 216866                       27.750000   
                                       217950                       17.516667   
                                       218728                       20.983333   
                                       218800                       46.516667   
                                       219513                        8.550000   

                                                         TEMPO P/ ALTA  
DS_MOTIVO_ALTA                         CD_PESSOA_FISICA                 
Alta Curado                            21726                129.533333  
                                       32416                134.200000  
                                       46850                224.450000  
                                       83799                269.866667  
                                       93614                 59.983333  
...                                                                ...  
Transferido para outro Estabelecimento 216866               297.866667  
                                       217950               212.550000  
                                       218728               912.000000  
                                       218800               417.916667  
                                       219513               455.616667  

[38948 rows x 3 columns]

In [31]:
df_tempos_p_alta = df_analysis.groupby(by=['DS_MOTIVO_ALTA'])[['TEMPO ESPERA', 'TEMPO P/ ATENDIMENTO',
       'TEMPO P/ ALTA']].mean()

Podemos ver que o maior tempo internado / atendimento e sobre "Transferido para outro estabelecimento" isso significar que o hostpital nao pode ou nao deve tratar esse caso

In [32]:
df_tempos_p_alta.sort_values(by='TEMPO P/ ALTA', ascending=False)

,TEMPO ESPERA,TEMPO P/ ATENDIMENTO,TEMPO P/ ALTA
DS_MOTIVO_ALTA,,,
Transferido para outro Estabelecimento,10.694372,25.651515,343.525541
Internação de Paciente Ambulatorial,9.544848,30.034805,228.547085
Alta a Pedido,10.643723,35.082143,212.202489
Por Óbito - Com Declaração de óbito fornecida pelo medico assistente,12.358333,53.055556,203.083333
Alta Melhorado,10.397837,37.665260,176.148281
Alta Curado,8.400000,34.815476,159.765476
Alta por Evasão,10.640252,53.272642,135.893396
Alta por Outros motivos,14.690972,80.683160,117.752083
Alta com previsão de retorno para acompanhamento do paciente,10.912500,35.230833,75.140000


In [45]:
df_analysis['DataIngestao'] = datetime.today()

In [46]:
df_analysis.to_parquet("DadosHospitalFinal.pq")

# Aqui fazemos o codigo automatizado a partir daqui sera considerado para tal. Em cima temos somente analise

In [47]:
historico = pd.read_parquet("DadosHospitalFinal.pq")

In [48]:
from datetime import datetime
df_ingest=df_analysis.copy()
df_ingest['DataIngestao'] = datetime.today()

In [51]:
pd.concat([df_ingest,historico]).sort_values(by='DataIngestao',ascending=False).drop_duplicates(subset='NR_ATENDIMENTO',keep='first')

,NR_ATENDIMENTO,DT_ENTRADA,DT_INICIO_ATENDIMENTO,DT_ATEND_MEDICO,DT_FIM_TRIAGEM,NR_SEQ_TRIAGEM,DS_SENHA_QMATIC,DT_MEDICACAO,NR_SEQ_CLASSIFICACAO,CD_MOTIVO_ALTA,...,NR_SEQ_PAC_SENHA_FILA,IE_CLINICA,DT_RECEBIMENTO_SENHA,DT_ATUALIZACAO,DS_FILA,DS_LETRA_VERIFICACAO,TEMPO ESPERA,TEMPO P/ ATENDIMENTO,TEMPO P/ ALTA,DataIngestao
0,188005,2023-06-22 14:19:51,2023-06-22 14:39:12,2023-06-22 15:36:45,2023-06-22 14:43:31,4.0,A454,NaN,8.0,12.0,...,174442.0,1,2023/06/22 14:17:00,2020-03-18 14:24:15,PS - Medicação/Procedimentos,CP,19.350000,76.900000,179.133333,2024-05-02 19:28:17.835715
63833,154734,2022-11-20 19:47:48,2022-11-20 19:58:28,2022-11-20 21:00:39,2022-11-20 20:01:11,4.0,SG1288,2022/11/20 21:13:35,17.0,12.0,...,168132.0,1,2022/11/20 19:46:04,2020-03-18 14:24:15,PS - Medicação/Procedimentos,CP,10.666667,72.850000,55.583333,2024-05-02 19:28:17.835715
63842,149351,2022-10-17 11:34:09,2022-10-17 11:41:08,2022-10-17 12:09:27,2022-10-17 11:43:23,4.0,SG1299,NaN,17.0,12.0,...,161015.0,1,2022/10/17 11:33:26,2020-03-18 14:24:15,PS - Medicação/Procedimentos,CP,6.983333,35.300000,58.850000,2024-05-02 19:28:17.835715
63841,149327,2022-10-17 09:52:46,2022-10-17 10:12:38,2022-10-17 10:17:34,2022-10-17 10:14:53,4.0,SG1294,2022/10/17 10:29:46,17.0,12.0,...,160980.0,1,2022/10/17 09:48:01,2020-03-18 14:24:15,PS - Medicação/Procedimentos,CP,19.866667,24.800000,169.683333,2024-05-02 19:28:17.835715
63840,189815,2023-07-04 08:10:00,2023-07-04 08:14:28,2023-07-04 08:30:35,2023-07-04 08:20:07,4.0,PF327,2023/07/04 08:41:15,8.0,12.0,...,215295.0,1,2023/07/04 08:07:54,2020-03-18 14:24:15,PS - Medicação/Procedimentos,CP,4.466667,20.583333,154.633333,2024-05-02 19:28:17.835715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31875,122999,2022-05-21 17:28:54,2022-05-21 17:58:06,2022-05-21 18:20:27,2022-05-21 18:01:17,4.0,NaN,NaN,17.0,12.0,...,125263.0,1,NaN,2020-03-18 14:24:15,PS - Medicação/Procedimentos,CP,29.200000,51.550000,64.483333,2024-05-02 19:28:17.835715
31873,136964,2022-08-10 06:10:18,2022-08-10 06:12:24,2022-08-10 06:41:06,2022-08-10 06:20:04,3.0,NM482,2022/08/10 06:32:45,8.0,26.0,...,144201.0,1,2022/08/10 06:09:29,2023-04-03 13:38:57,PS - Ambulância,AM,2.100000,30.800000,428.116667,2024-05-02 19:28:17.835715
31877,147115,2022-10-01 15:43:31,2022-10-01 15:44:39,2022-10-01 15:53:15,2022-10-01 15:49:08,4.0,PF344,2022/10/01 16:03:09,8.0,12.0,...,157978.0,1,2022/10/01 15:42:11,2020-03-18 14:24:15,PS - Medicação/Procedimentos,CP,1.133333,9.733333,105.450000,2024-05-02 19:28:17.835715
31878,178267,2023-04-26 08:30:33,2023-04-26 08:35:07,2023-04-26 08:38:46,2023-04-26 08:36:38,4.0,NM539,2023/04/26 09:18:31,8.0,12.0,...,199707.0,1,2023/04/26 08:28:45,2020-03-18 14:24:15,PS - Medicação/Procedimentos,CP,4.566667,8.216667,309.133333,2024-05-02 19:28:17.835715
